In [1]:
import openpyxl
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import accuracy_score
from numpy.linalg import norm
from sklearn.model_selection import LeaveOneOut
from decimal import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import math
import xgboost as xgb
from xgboost import XGBClassifier

number_runnings = 50
water = pd.read_csv("water.csv")
water['ph'] = water['ph'].replace(to_replace=np.nan,value = water['ph'].median())
water['Hardness'] = water['Hardness'].replace(to_replace=np.nan,value = water['Hardness'].median())
water['Solids'] = water['Solids'].replace(to_replace=np.nan,value = water['Solids'].median())
water['Chloramines'] = water['Chloramines'].replace(to_replace=np.nan,value = water['Chloramines'].median())
water['Sulfate'] = water['Sulfate'].replace(to_replace=np.nan,value = water['Sulfate'].median())
water['Conductivity'] = water['Conductivity'].replace(to_replace=np.nan,value = water['Conductivity'].median())
water['Organic_carbon'] = water['Organic_carbon'].replace(to_replace=np.nan,value = water['Organic_carbon'].median())
water['Trihalomethanes'] = water['Trihalomethanes'].replace(to_replace=np.nan,value = water['Trihalomethanes'].median())
water['Turbidity'] = water['Turbidity'].replace(to_replace=np.nan,value = water['Turbidity'].median())

water['Outcome'].value_counts()
Y = water['Outcome']
target_names=["0", "1"]  

X = water[['ph',	'Hardness',	'Solids',	'Chloramines',	'Sulfate',	'Conductivity',	'Organic_carbon',	'Trihalomethanes',	'Turbidity']]
maximums = np.amax(np.array(X), axis=0)
X_featurenames = X.columns
noOfInstances = water.shape[0]
print(noOfInstances)
df = pd.DataFrame(water)
corr = df.corrwith(df["Outcome"])
print(corr)

counter = Counter(Y)
print(counter[0])
print(counter[1])

feature_names = ['ph',	'Hardness',	'Solids',	'Chloramines',	'Sulfate',	'Conductivity',	'Organic_carbon',	'Trihalomethanes',	'Turbidity']
results = pd.read_excel('resultsRankingsWaterMSPMS_NoMinMax.xlsx')
noOfIterations = 20

model = XGBClassifier()
# fit the model
model.fit(X, Y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i in range(len(feature_names)):
    print(importance[i])

df_LIME = pd.read_excel('rankingsWaterLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]
W_LIME = []

df_Anchor = pd.read_excel('rankingsWaterAnchor.xlsx')
df_Anchor = df_Anchor[df_Anchor.columns[1:]]
W_Anchor = []

df_TreeShap = pd.read_excel('rankingsWaterTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_KernelShap = pd.read_excel('rankingsWaterKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
W_Shap = []
W_all = []

for instance in range (0, noOfInstances):
    m = pd.DataFrame(columns = ['A', 'L', 'S'])
    line_Anchor  = [results.iloc[instance, 1], results.iloc[instance, 2], results.iloc[instance, 3]]
    m.loc[0] = line_Anchor
    line_LIME  = [results.iloc[instance, 4], results.iloc[instance, 5], results.iloc[instance, 6]]
    m.loc[1] = line_LIME
    line_Shap  = [results.iloc[instance, 7], results.iloc[instance, 8], results.iloc[instance, 9]]
    m.loc[2] = line_Shap
    A = []
    for i in range(0, 3):
        s = 0
        for j in range(0, 3):
            s = s + m.iloc[i, j] + m.iloc[j, i]
        s = s / 6
        A.append(s)
       
    data_LIME = []
    data_Anchor = []
    data_Shap = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
        data_Anchor.append(df_Anchor.loc[instance * noOfIterations + i].values.flatten().tolist())
        
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
        
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    w_LIME = []
    M_Anchor  = pd.DataFrame(data_Anchor, columns = feature_names)
    w_Anchor = []
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    w_Shap = []
    A = [1, 1, 1]
    for i in range(noOfIterations):
        x = [0] * len(feature_names)
        for k in range(0, 3):
            x = x + A[k] * M_LIME.loc[i].values.flatten()
        y = sum(A)
        w = [0] * len(feature_names)
        for k in range(len(feature_names)):
            w[k] = x[k] / y
        w_LIME.append(w)

        x = [0] * len(feature_names)
        for k in range(0, 3):
            x = x + A[k] * M_Anchor.loc[i].values.flatten()
        y = sum(A)
        w = [0] * len(feature_names)
        for k in range(len(feature_names)):
            w[k] = x[k] / y
        w_Anchor.append(w)

        x = [0] * len(feature_names)
        for k in range(0, 3):
            x = x + A[k] * M_Shap.loc[i].values.flatten()
        y = sum(A)
        w = [0] * len(feature_names)
        for k in range(len(feature_names)):
            w[k] = x[k] / y
        w_Shap.append(w)
        
    averageLIME = np.mean(w_LIME, axis=0)
    W_LIME.append(averageLIME)
    averageAnchor = np.mean(w_Anchor, axis=0)
    W_Anchor.append(averageAnchor)
    averageShap = np.mean(w_Shap, axis=0)
    W_Shap.append(averageShap)
    averageMatrix = [averageLIME]
    average = np.mean(averageMatrix, axis=0)
    sum_average = sum(average)
    sum_corr = sum(corr) - 1
    for i in range(0, len(feature_names)):
        #average[i] = (average[i] / sum_average + corr.iloc[i] / sum_corr) / 2
        #average[i] = average[i] * corr.iloc[i]
        #average[i] = average[i] * (corr.iloc[i] / sum_corr)
        #average[i] = average[i] + corr.iloc[i]
        average[i] = average[i] 
    W_all.append(average)
    # print(averageLIME)
    # print(averageAnchor)
    # print(averageShap)
    # print(average)
    # print("============")
average_all = np.mean(W_all, axis=0)
print(average_all)


def compute_weights(distance):
    weights = []
    for i in range(len(distance)): 
        weights.append(norm(W_all[i]) * distance[i])
    return weights

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
    
# knn = KNeighborsClassifier(n_neighbors = 5)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))

#X2 = scaler.fit_transform(X)
scale_pos_weight = counter[0] / counter[1]
model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
# model.fit(X_train, Y_train)
# pred = model.predict(X_test)
# accuracy = accuracy_score(Y_test, pred)
# print("Accuracy:", accuracy)
scores = cross_val_score(model, X, Y, cv = 10, scoring='roc_auc')
print(np.mean(scores))


for i in range(noOfInstances):
    for j in range(len(feature_names)):
        X.iloc[i, j] = X.iloc[i, j] * math.sqrt(W_all[i][j])


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X = scaler.fit_transform(X)
#knn = KNeighborsClassifier(n_neighbors = 5, weights = 'uniform')
model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
scores = cross_val_score(model, X, Y, cv = 10, scoring='roc_auc')
print(np.mean(scores))


# knn = KNeighborsClassifier(n_neighbors = 5)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))
    
# knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))
# print("---------------")

# knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
# scores = cross_val_score(knn, X, Y, cv=20)
# print(scores)
# print(np.mean(scores))

# cv = LeaveOneOut()
# s = 0
# for train_ix, test_ix in cv.split(X):
#     knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
#     X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
#     Y_train, Y_test = Y[train_ix], Y[test_ix]
#     knn.fit(X_train, Y_train)  
#     pred = knn.predict(X_test)
#     accuracy_classifier = accuracy_score(Y_test, pred)
#     s = s + accuracy_classifier
# print(s / len(X))

def euclidian_distance(a, b):
    dim = len(a)
    distance = 0
    for i in range(dim):
        distance += abs(a[i] - b[i])**2  
    distance = math.sqrt(distance)    
    return distance


def knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k):
    
    from collections import Counter
    Y_hat_test = []

    for test_point in X_test:
        distances = []
        for train_point in X_train:
            dim = len(train_point)
            distance = 0
            for i in range(dim):
                distance = distance + math.sqrt((train_point[i] - test_point[i]) * (train_point[i] - test_point[i]))
            distances.append(distance)

        df_dists = pd.DataFrame(data = distances, columns = ['dist'], 
                                index = Y_train.index)

        df_nn = df_dists.sort_values(by=['dist'], axis = 0)[:k]

        counter = Counter(Y_train[df_nn.index])

        prediction = counter.most_common()[0][0]

        Y_hat_test.append(prediction)
        
    return Y_hat_test



def knn_predict_weighted(X_train, X_test, Y_train, Y_test, weights, k):
    
    from collections import Counter
    Y_hat_test = []

    for test_point in X_test:
        distances = []
        for train_point in X_train:
            dim = len(train_point)
            distance = 0
            for i in range(dim):
                distance = distance + math.sqrt(weights[i] * (train_point[i] - test_point[i]) * (train_point[i] - test_point[i]))
            distances.append(distance)

        df_dists = pd.DataFrame(data = distances, columns = ['dist'], 
                                index = Y_train.index)

        df_nn = df_dists.sort_values(by=['dist'], axis = 0)[:k]

        counter = Counter(Y_train[df_nn.index])

        prediction = counter.most_common()[0][0]

        Y_hat_test.append(prediction)
        
    return Y_hat_test

outputs = pd.DataFrame(columns = ['ACC_WL', 'F1_WL', 'ROC_WL'])
acc_non_weighted = 0
acc_weighted = 0
f1_non_weighted = 0
f1_weighted = 0
roc_non_weighted = 0
roc_weighted = 0
for i in range(number_runnings):
    output = []
    #print('------------------')
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    Y_hat_test = knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k=5)
    #print(f1_score(Y_test, Y_hat_test))
    acc_non_weighted += accuracy_score(Y_test, Y_hat_test)
    output.append(accuracy_score(Y_test, Y_hat_test))
    f1_non_weighted += f1_score(Y_test, Y_hat_test)
    output.append(f1_score(Y_test, Y_hat_test))
    roc_non_weighted += roc_auc_score(Y_test, Y_hat_test)
    output.append(roc_auc_score(Y_test, Y_hat_test))
    

    outputs.loc[i] = output


acc_non_weighted = acc_non_weighted / number_runnings
acc_weighted = acc_weighted / number_runnings
f1_non_weighted = f1_non_weighted / number_runnings
f1_weighted = f1_weighted / number_runnings
roc_non_weighted = roc_non_weighted / number_runnings
roc_weighted = roc_weighted / number_runnings
output = []
output.append(acc_non_weighted)
output.append(f1_non_weighted)
output.append(roc_non_weighted)

outputs.loc[number_runnings] = output

outputs.to_excel("Rankings_accuraciesWaterMSPMS_NoMinMax_LIME_" + str(number_runnings) + ".xlsx")

# cv = LeaveOneOut()
# s_non_weighted = 0
# s_weighted = 0
# for train_ix, test_ix in cv.split(X):
#     X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
#     Y_train, Y_test = Y[train_ix], Y[test_ix]
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
#     Y_hat_test_non_weighted = knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k=5)
#     s_non_weighted = s_non_weighted + accuracy_score(Y_test, Y_hat_test_non_weighted)
#     Y_hat_test_weighted = knn_predict_weighted(X_train, X_test, Y_train, Y_test, k=5)
#     s_weighted = s_weighted + accuracy_score(Y_test, Y_hat_test_non_weighted)
# print(s_non_weighted / len(X))
# print(s_weighted / len(X))

3276
ph                -0.003014
Hardness          -0.013837
Solids             0.033743
Chloramines        0.023779
Sulfate           -0.020476
Conductivity      -0.008128
Organic_carbon    -0.030001
Trihalomethanes    0.006887
Turbidity          0.001581
Outcome            1.000000
dtype: float64
1998
1278
0.14890973
0.11521123
0.11634465
0.119793914
0.12383043
0.09632984
0.09617217
0.09755744
0.085850604
[8.26410256 5.11710928 4.68545482 4.17300061 8.24271978 3.5729243
 4.42493895 3.6508547  2.86889499]
0.6090413589848851
0.5923618524771991
